<a href="https://colab.research.google.com/github/Surya-prakash-v/MLAI/blob/master/kerasDetection.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

####Analytics Vidya</b>
###Face detection and counting</p></b>
<p>People detection and head counting is one of the classical albeit challenging computer vision application. For this problem, given a group selfie/photo, you are required to count the number of heads present in the picture. You are provided with a training set of images with coordinates of bounding box and head count for each image and need to predict the headcount for each image in the test set.</p></br>

In [ ]:
from google.colab import drive
import zipfile as zip1
import os
drive.mount('/content/drive')
zip_ref = zip1.ZipFile('./drive/My Drive/Colab Notebooks/AnalyticsVidya/train_HNzkrPW (1).zip', 'r')
zip_ref.extractall('train/')
zip_ref.close()

Mounted at /content/drive


In [ ]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator
import pandas as pd

IMG_SIZE = 128
BATCH_SIZE = 64
traindatagen = ImageDataGenerator(rescale=1./255)
traincsv = pd.read_csv('train/train.csv',dtype={'Name': str,'HeadCount':int})
trainFaceMaskcsv = pd.read_csv('train/bbox_train.csv',dtype={'Name': str,'width':float,'height':float,'xmin':float,'ymin':float,'xmax':float,'ymax':float})
trainFaceMaskcsv['xmin'] = trainFaceMaskcsv['xmin']*IMG_SIZE/trainFaceMaskcsv['width']
trainFaceMaskcsv['xmax'] = trainFaceMaskcsv['xmax']*IMG_SIZE/trainFaceMaskcsv['width']
trainFaceMaskcsv['ymin'] = trainFaceMaskcsv['ymin']*IMG_SIZE/trainFaceMaskcsv['height']
trainFaceMaskcsv['ymax'] = trainFaceMaskcsv['ymax']*IMG_SIZE/trainFaceMaskcsv['height']

traingenerator = traindatagen.flow_from_dataframe(traincsv, directory='train/image_data/',x_col='Name', y_col=['Name','HeadCount'],target_size=(IMG_SIZE,IMG_SIZE), color_mode='rgb',class_mode='raw', batch_size=BATCH_SIZE,shuffle=False, seed=10)

Found 5733 validated image filenames.


In [ ]:
import numpy as np
def get_iou(a, epsilon=1e-5):
    a[:,9] = np.maximum(a[:,1], a[:,5])
    a[:,10] = np.maximum(a[:,2], a[:,6])
    a[:,11] = np.minimum(a[:,3], a[:,7])
    a[:,12] = np.minimum(a[:,4], a[:,8])
    overlap = np.multiply(np.maximum(0,np.subtract(a[:,11],a[:,9])),np.maximum(0,np.subtract(a[:,12],a[:,10])))
    area_a = np.multiply(np.subtract(a[:,3],a[:,1]),np.subtract(a[:,4],a[:,2]))
    area_b = np.multiply(np.subtract(a[:,7],a[:,5]),np.subtract(a[:,8],a[:,6]))
    area_combined = np.subtract(area_a+area_b,overlap)+epsilon
    a[:,13] = np.divide(overlap,area_combined)*IMG_SIZE
    a[np.where(a[:,13] > 0),14] = IMG_SIZE 
    a[np.where(a[:,13] <= 0),14] = 0
    return a

In [ ]:
def get_iou_tf(a, epsilon=1e-5):
  a1max = tf.maximum(a[:,0], a[:,4])
  a2max = tf.maximum(a[:,1], a[:,5])
  a3min = tf.minimum(a[:,2], a[:,6])
  a4min = tf.minimum(a[:,3], a[:,7])
  overlap = tf.multiply(tf.maximum(0.0,tf.subtract(a3min,a1max)),tf.maximum(0.0,tf.subtract(a4min,a2max)))
  area_a = tf.multiply(tf.subtract(a[:,2],a[:,0]),tf.subtract(a[:,3],a[:,1]))
  area_b = tf.multiply(tf.subtract(a[:,6],a[:,4]),tf.subtract(a[:,7],a[:,5]))
  area_combined = tf.add(tf.subtract(tf.add(area_a,area_b),overlap),epsilon)
  return tf.divide(overlap,area_combined)

In [ ]:
def getAnchorBasedSlidingWindow(image_size,anchor):
  array = []
  for i in range(0,image_size,anchor):
    for j in range(0,image_size,anchor):
      array.append([i,j,i+anchor,j+anchor])
  return array

def getallSlidingwindows(image_size):
  window8 = np.array(getAnchorBasedSlidingWindow(image_size,8))
  window16 = np.array(getAnchorBasedSlidingWindow(image_size,16))
  window32 = np.array(getAnchorBasedSlidingWindow(image_size,32))
  window64 = np.array(getAnchorBasedSlidingWindow(image_size,64))

  return np.vstack([np.hstack([np.ones(shape=(window8.shape[0],1))*8,window8]),
            np.hstack([np.ones(shape=(window16.shape[0],1))*16,window16]),
            np.hstack([np.ones(shape=(window32.shape[0],1))*32,window32]),
            np.hstack([np.ones(shape=(window64.shape[0],1))*64,window64])])

In [ ]:
def get_faceMasks(traingenerator):
  while True:
    batch_x, batch_y = next(traingenerator)
    allwin = getallSlidingwindows(IMG_SIZE)
    facemasks8 = np.zeros(shape=(batch_y.shape[0],int(IMG_SIZE/8),int(IMG_SIZE/8),7))
    facemasks16 = np.zeros(shape=(batch_y.shape[0],int(IMG_SIZE/16),int(IMG_SIZE/16),7))
    facemasks32 = np.zeros(shape=(batch_y.shape[0],int(IMG_SIZE/32),int(IMG_SIZE/32),7))
    facemasks64 = np.zeros(shape=(batch_y.shape[0],int(IMG_SIZE/64),int(IMG_SIZE/64),7))
    onestemp = np.ones(shape=(allwin.shape[0]),dtype=np.float64)
    for i in range(batch_y.shape[0]):
      faceList = trainFaceMaskcsv.where(trainFaceMaskcsv['Name']==batch_y[i][0])
      faceList = faceList[faceList.Name.notnull()]
      arr = []
      for face in range(faceList['Name'].count()):
        arr.append(np.concatenate([allwin,np.stack([onestemp*faceList['xmin'].iloc[face],onestemp*faceList['ymin'].iloc[face],onestemp*faceList['xmax'].iloc[face],onestemp*faceList['ymax'].iloc[face],
                                    onestemp*0,onestemp*0,onestemp*0,onestemp*0,onestemp*0,onestemp*0,onestemp*faceList['Name'].count()],axis=1)],axis=1))
      ious = get_iou(np.array(arr).reshape(faceList['Name'].count()*allwin.shape[0],16))
      ious2 = pd.DataFrame(ious)
      idx = ious2.loc[ious2.reset_index().groupby([0,1,2,3,4])[13].idxmax()]
      idxarray = np.array(idx)
      facemasks8[i] = np.reshape((idxarray[idxarray[:,0]==8][:,1:16]/IMG_SIZE)[:,[8,9,10,11,12,13,14]],newshape=(int(IMG_SIZE/8),int(IMG_SIZE/8),7))
      facemasks16[i] = np.reshape((idxarray[idxarray[:,0]==16][:,1:16]/IMG_SIZE)[:,[8,9,10,11,12,13,14]],newshape=(int(IMG_SIZE/16),int(IMG_SIZE/16),7))
      facemasks32[i] = np.reshape((idxarray[idxarray[:,0]==32][:,1:16]/IMG_SIZE)[:,[8,9,10,11,12,13,14]],newshape=(int(IMG_SIZE/32),int(IMG_SIZE/32),7))
      facemasks64[i] = np.reshape((idxarray[idxarray[:,0]==64][:,1:16]/IMG_SIZE)[:,[8,9,10,11,12,13,14]],newshape=(int(IMG_SIZE/64),int(IMG_SIZE/64),7))
    yield batch_x, [facemasks8.astype('float32'),facemasks16.astype('float32'),facemasks32.astype('float32'),facemasks64.astype('float32')]

In [ ]:
import tensorflow as tf
from tensorflow.python.keras.layers import (Flatten,LeakyReLU,AveragePooling2D,concatenate,
                                            Reshape,Activation,Input, Conv2D, MaxPooling2D,
                                            BatchNormalization, GlobalAveragePooling2D,SpatialDropout2D)
def resNetBlock(input_image,num):

  layer1 = Conv2D(64, 3, strides=(1,1), padding='same', use_bias=False,name ='rone'+str(num))(input_image)
  layer1 = BatchNormalization()(layer1)
  layer1 = LeakyReLU(alpha=0.1)(layer1)

  layerp = Conv2D(128, 3, strides=(1,1), padding='same', use_bias=False,name ='rp'+str(num))(layer1)
  layerp = BatchNormalization()(layerp)
  layerp = LeakyReLU(alpha=0.1)(layerp)

  layerq = Conv2D(256, 3, strides=(1,1), padding='same', use_bias=False,name ='rq'+str(num))(layerp)
  layerq = BatchNormalization()(layerq)
  layerq = LeakyReLU(alpha=0.1)(layerq)
  
  layer3 = Conv2D(16, 1, strides=(1,1), padding='same', use_bias=False,name='rtwo'+str(num))(layerq)
  layer3 = BatchNormalization()(layer3)
  layer3 = LeakyReLU(alpha=0.1)(layer3)
    
  return layer3  

def build_model(img_size,img_ch):
  input_sh = Input(shape=(img_size,img_size,img_ch)) 

  layer_1 = Conv2D(16, 3, strides=(1,1), padding='same', use_bias=False,name='one')(input_sh)
  layer_1 = BatchNormalization()(layer_1)
  layer_1 = LeakyReLU(alpha=0.1)(layer_1)
  
  layer_2 = Conv2D(32, 3, strides=(1,1), padding='same', use_bias=False,name='two')(layer_1)
  layer_2 = BatchNormalization()(layer_2)
  layer_2 = LeakyReLU(alpha=0.1)(layer_2)
 
  resNetBlock1 = resNetBlock(layer_2,1)
  concact1 = concatenate([resNetBlock1, layer_2])
  layer_l = LeakyReLU(alpha=0.1)(concact1)
  maxpool1 = MaxPooling2D(pool_size=(2, 2))(layer_l)

  resNetBlock2 = resNetBlock(maxpool1,2)
  concact2 = concatenate([resNetBlock2, maxpool1])
  layer_l2 = LeakyReLU(alpha=0.1)(concact2)
  maxpool2 = MaxPooling2D(pool_size=(2, 2))(layer_l2)

  resNetBlock3 = resNetBlock(maxpool2,3)
  concact3 = concatenate([resNetBlock3, maxpool2])
  layer_l3 = LeakyReLU(alpha=0.1)(concact3)
  maxpool3 = MaxPooling2D(pool_size=(2, 2))(layer_l3)
  
  resNetBlock4 = resNetBlock(maxpool3,4)
  outputw = Conv2D(7, 1, strides=(1,1), padding='same', use_bias=False,name='six')(resNetBlock4)
  concact4 = concatenate([resNetBlock4, maxpool3])
  layer_l4 = LeakyReLU(alpha=0.1)(concact4)
  maxpool4 = MaxPooling2D(pool_size=(2, 2))(layer_l4)
  
  resNetBlock5 = resNetBlock(maxpool4,5)
  outputk = Conv2D(7, 1, strides=(1,1), padding='same', use_bias=False,name='seven')(resNetBlock5)
  concact5 = concatenate([resNetBlock5, maxpool4])
  layer_l5 = LeakyReLU(alpha=0.1)(concact5)
  maxpool5 = MaxPooling2D(pool_size=(2, 2))(layer_l5)
  
  resNetBlock6 = resNetBlock(maxpool5,6)
  concact6 = concatenate([resNetBlock6, maxpool5])
  layer_l6 = LeakyReLU(alpha=0.1)(concact6)
  layer_ant = Conv2D(7, (1,1), strides=(1,1), padding='same', use_bias=False, name='eight')(layer_l6)
  
  layer_plus1 = Conv2D(64, (3,3), strides=(1,1),  use_bias=False, name='nine')(layer_ant)
  layer_plus2 = Conv2D(7, (3,3), strides=(1,1),padding='same', use_bias=False, name='ten')(layer_plus1)
  model = tf.keras.Model(input_sh,outputs=[outputw,outputk,layer_ant,layer_plus2])
  return model

In [ ]:
model = build_model(IMG_SIZE,3)
model.summary()

Model: "model"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None, 128, 128, 3) 0                                            
__________________________________________________________________________________________________
one (Conv2D)                    (None, 128, 128, 16) 432         input_1[0][0]                    
__________________________________________________________________________________________________
batch_normalization (BatchNorma (None, 128, 128, 16) 64          one[0][0]                        
__________________________________________________________________________________________________
leaky_re_lu (LeakyReLU)         (None, 128, 128, 16) 0           batch_normalization[0][0]        
______________________________________________________________________________________________

In [ ]:
resolver = tf.distribute.cluster_resolver.TPUClusterResolver(tpu='grpc://' + os.environ['COLAB_TPU_ADDR'])
tf.config.experimental_connect_to_cluster(resolver)
tf.tpu.experimental.initialize_tpu_system(resolver)
print("All devices: ", tf.config.list_logical_devices('TPU'))
strategy = tf.distribute.TPUStrategy(resolver)

INFO:tensorflow:Initializing the TPU system: grpc://10.24.106.250:8470


INFO:tensorflow:Initializing the TPU system: grpc://10.24.106.250:8470


INFO:tensorflow:Clearing out eager caches


INFO:tensorflow:Clearing out eager caches


INFO:tensorflow:Finished initializing TPU system.


...

INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:worker/replica:0/task:0/device:TPU:7, TPU, 0, 0)


INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:worker/replica:0/task:0/device:TPU_SYSTEM:0, TPU_SYSTEM, 0, 0)


INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:worker/replica:0/task:0/device:TPU_SYSTEM:0, TPU_SYSTEM, 0, 0)


INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:worker/replica:0/task:0/device:XLA_CPU:0, XLA_CPU, 0, 0)


INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:worker/replica:0/task:0/device:XLA_CPU:0, XLA_CPU, 0, 0)


In [ ]:
import tensorflow_addons as tfa
with strategy.scope():
  gl = tfa.losses.GIoULoss(reduction=tf.keras.losses.Reduction.NONE)
  bce = tf.keras.losses.BinaryCrossentropy(reduction=tf.keras.losses.Reduction.NONE,from_logits=True)
  mse = tf.keras.losses.MeanSquaredError(reduction=tf.keras.losses.Reduction.NONE)
  mseMetric = tf.keras.metrics.MeanSquaredError()
  classifiMetric = tf.keras.metrics.BinaryCrossentropy()

  def reshape_things(labels, predictions):
    shape1 = (labels[0].shape[0]*labels[0].shape[1]*labels[0].shape[2],labels[0].shape[3])
    shape2 = (labels[1].shape[0]*labels[1].shape[1]*labels[1].shape[2],labels[1].shape[3])
    shape3 = (labels[2].shape[0]*labels[2].shape[1]*labels[2].shape[2],labels[2].shape[3])
    shape4 = (labels[3].shape[0]*labels[3].shape[1]*labels[3].shape[2],labels[3].shape[3])
    labels_shaped = tf.concat([tf.reshape(labels[0],shape=shape1),tf.reshape(labels[1],shape=shape2),tf.reshape(labels[2],shape=shape3),tf.reshape(labels[3],shape=shape4)],axis=0)
    predictions_shaped = tf.concat([tf.reshape(predictions[0],shape=shape1),tf.reshape(predictions[1],shape=shape2),tf.reshape(predictions[2],shape=shape3),tf.reshape(predictions[3],shape=shape4)],axis=0)
    classifylabel = tf.reshape(labels_shaped[:,5],shape=(labels_shaped[:,5].shape[0],1))
    classifypred = tf.reshape(predictions_shaped[:,5],shape=(predictions_shaped[:,5].shape[0],1))
    classifylabelnorm = tf.divide(tf.subtract(classifylabel,tf.reduce_min(classifylabel)),tf.subtract(tf.reduce_max(classifylabel),tf.reduce_min(classifylabel)))
    classifyprednorm = tf.divide(tf.subtract(classifypred,tf.reduce_min(classifypred)),tf.subtract(tf.reduce_max(classifypred),tf.reduce_min(classifypred)))
    bc1 = tf.concat([classifylabelnorm,tf.abs(tf.subtract(classifylabelnorm,tf.ones(classifylabelnorm.shape)))],axis=1)
    bc2 = tf.concat([classifyprednorm,tf.abs(tf.subtract(classifyprednorm,tf.ones(classifyprednorm.shape)))],axis=1)
    mselabel = tf.reshape(labels_shaped[:,6],shape=(labels_shaped[:,6].shape[0],1))
    msepred = tf.reshape(predictions_shaped[:,6],shape=(predictions_shaped[:,6].shape[0],1))
    ioulabel = tf.reshape(labels_shaped[:,0:4],shape=(labels_shaped[:,0:4].shape[0],4))
    ioupred = tf.reshape(predictions_shaped[:,0:4],shape=(predictions_shaped[:,0:4].shape[0],4))
    return bc1,bc2,mselabel,msepred,ioulabel,ioupred,classifylabel
  
  def compute_loss(labels, predictions):
    bc1,bc2,mselabel,msepred,ioulabel,ioupred,classifylabel = reshape_things(labels, predictions)
    p1 = bce(bc1,bc2)
    p2 = mse(mselabel,msepred)
    p3 = gl(ioulabel,ioupred)
    pp11 = tf.nn.compute_average_loss(p1)
    pp22 = tf.nn.compute_average_loss(p2 *classifylabel)
    pp33 = tf.nn.compute_average_loss(p3* classifylabel)
    return pp11,pp22,pp33

  def custom_metric(labels,predictions):
    bc1,bc2,mselabel,msepred,ioulabel,ioupred,classifylabel = reshape_things(labels, predictions)
    classifiMetric.update_state(bc1,bc2)
    mseMetric.update_state(mselabel,msepred)
    iou = get_iou_tf(tf.concat([ioulabel,ioupred],1))
    return tf.math.reduce_mean(iou)
    
  def custom_metric_reset():
    return classifiMetric.reset_states(),mseMetric.reset_states()
  def custom_metric_result():
    return classifiMetric.result(),mseMetric.result()

  model = build_model(IMG_SIZE,3)
  optimizer = tf.keras.optimizers.Adam()

In [ ]:
from tensorflow.data import Dataset
from tensorflow.python.keras.callbacks import ModelCheckpoint
def train_step(input):
  image,[label1,label2,label3,label4] = input
  with tf.GradientTape() as tape:
    predictions1,predictions2,predictions3,predictions4 = model(image, training=True)
    l1,l2,l3 = compute_loss([label1,label2,label3,label4],[predictions1,predictions2,predictions3,predictions4])
  gradients = tape.gradient([l1,l2,l3], model.trainable_variables)
  updated_vars = optimizer.apply_gradients(list(zip(gradients,model.trainable_variables)))
  iou = custom_metric([label1,label2,label3,label4],[predictions1,predictions2,predictions3,predictions4])
  return l1,l2,l3,iou

@tf.function
def distributed_train_epoch(xx,yy):
  [yy1,yy2,yy3,yy4] = yy
  l1,l2,l3,iou = strategy.run(train_step,args=((xx,[yy1,yy2,yy3,yy4]),))
  return strategy.reduce(tf.distribute.ReduceOp.SUM, l1, axis=None),strategy.reduce(tf.distribute.ReduceOp.SUM, l2, axis=None),strategy.reduce(tf.distribute.ReduceOp.SUM, l3, axis=None),strategy.reduce(tf.distribute.ReduceOp.SUM, iou, axis=None)

def custom_loop(initial_epoch,num_epochs):
  for epoch in range(initial_epoch,num_epochs):
    total_loss=0.0
    num_batches = 0
    total_metric = 0.0
    iou_per_epoch = 0.0
    for batch in range(int(5733/BATCH_SIZE)):
      num_batches +=1
      xx,[m1,m2,m3,m4] = next(get_faceMasks(traingenerator))
      l1,l2,l3,iou = distributed_train_epoch(xx,[m1,m2,m3,m4])
      total_loss += l1+l2+l3
      iou_per_epoch += iou
    f1,f2 = custom_metric_result()
    total_metric = (f1 +(iou_per_epoch/num_batches))/2 
    custom_metric_reset()
    template = ('Epoch: {} -> , Train Loss: {:.6f}, facecount_regress_mse:{:.6f}      Train Accuracy: {:.6f},classification Accuracy: {:.6f}, iou_accuracy: {:.6f}')
    print(template.format(epoch,total_loss/num_batches,f2, total_metric,f1,iou_per_epoch/num_batches))
  model.save('drive/My Drive/Colab Notebooks/AnalyticsVidya/FaceDetection/face_detectionTPU'+initial_epoch+'_'+num_epochs+'.hdf5')

In [ ]:
custom_loop(0,20)

Epoch: 0, Train Loss: 4810.513672, Train Accuracy: 0.399359,classification Accuracy: 0.736960, mean squared error:0.053617, iou_accuracy: 0.004070
Epoch: 1, Train Loss: 4214.025391, Train Accuracy: 0.254804,classification Accuracy: 0.490789, mean squared error:0.005711, iou_accuracy: 0.006554
Epoch: 2, Train Loss: 4182.578613, Train Accuracy: 0.255724,classification Accuracy: 0.489831, mean squared error:0.002568, iou_accuracy: 0.009525
Epoch: 3, Train Loss: 4175.518555, Train Accuracy: 0.284528,classification Accuracy: 0.531814, mean squared error:0.001624, iou_accuracy: 0.017809
Epoch: 4, Train Loss: 4171.503418, Train Accuracy: 0.320114,classification Accuracy: 0.577931, mean squared error:0.001306, iou_accuracy: 0.030495
Epoch: 5, Train Loss: 4162.186035, Train Accuracy: 0.334789,classification Accuracy: 0.576958, mean squared error:0.001161, iou_accuracy: 0.045730
Epoch: 6, Train Loss: 4153.070312, Train Accuracy: 0.352006,classification Accuracy: 0.568532, mean squared error:0.00

####Helper functions</b>

In [ ]:
custom_loop(20,120)

Epoch: 20, Train Loss: 4105.806152, Train Accuracy: 0.508834,classification Accuracy: 0.732696, mean squared error:0.000571, iou_accuracy: 0.142200
Epoch: 21, Train Loss: 4100.610840, Train Accuracy: 0.510800,classification Accuracy: 0.727033, mean squared error:0.000539, iou_accuracy: 0.147014
Epoch: 22, Train Loss: 4097.215820, Train Accuracy: 0.514508,classification Accuracy: 0.728691, mean squared error:0.000531, iou_accuracy: 0.149898
Epoch: 23, Train Loss: 4094.894775, Train Accuracy: 0.516278,classification Accuracy: 0.727587, mean squared error:0.000518, iou_accuracy: 0.152226
Epoch: 24, Train Loss: 4103.167480, Train Accuracy: 0.518133,classification Accuracy: 0.729872, mean squared error:0.000518, iou_accuracy: 0.152937
Epoch: 25, Train Loss: 4098.235840, Train Accuracy: 0.516633,classification Accuracy: 0.724149, mean squared error:0.000508, iou_accuracy: 0.154304
Epoch: 26, Train Loss: 4100.704102, Train Accuracy: 0.516586,classification Accuracy: 0.719071, mean squared err

KeyboardInterrupt: ignored

In [ ]:
with strategy.scope():
  #model = tf.keras.models.load_model('drive/My Drive/Colab Notebooks/AnalyticsVidya/FaceDetection/face_detectionTPU.hdf5')
  optimizer = tf.keras.optimizers.Adam(learning_rate=0.0001)
custom_loop(35,120)

Epoch: 35, Train Loss: 4082.916504, Train Accuracy: 0.537043,classification Accuracy: 0.737576, mean squared error:0.000423, iou_accuracy: 0.168044
Epoch: 36, Train Loss: 4090.511230, Train Accuracy: 0.540465,classification Accuracy: 0.738216, mean squared error:0.000473, iou_accuracy: 0.171120
Epoch: 37, Train Loss: 4090.035156, Train Accuracy: 0.540135,classification Accuracy: 0.737834, mean squared error:0.000493, iou_accuracy: 0.170971
Epoch: 38, Train Loss: 4090.602539, Train Accuracy: 0.542844,classification Accuracy: 0.741490, mean squared error:0.000501, iou_accuracy: 0.171849
Epoch: 39, Train Loss: 4095.536865, Train Accuracy: 0.544821,classification Accuracy: 0.744912, mean squared error:0.000494, iou_accuracy: 0.172118
Epoch: 40, Train Loss: 4094.014648, Train Accuracy: 0.543019,classification Accuracy: 0.739735, mean squared error:0.000491, iou_accuracy: 0.172906
Epoch: 41, Train Loss: 4085.377930, Train Accuracy: 0.541666,classification Accuracy: 0.732774, mean squared err

In [ ]:
model.save('drive/My Drive/Colab Notebooks/AnalyticsVidya/FaceDetection/face_detectionTPU2.hdf5')

In [ ]:
from matplotlib import pyplot as plt
import matplotlib.patches as patches
x_train,[m1,m2,m3,m4] = next(get_faceMasks(traingenerator))
newm1 = np.reshape(m1,newshape=(m1.shape[0],int((IMG_SIZE*IMG_SIZE)/(8*8)),7))
newm2 = np.reshape(m2,newshape=(m2.shape[0],int((IMG_SIZE*IMG_SIZE)/(16*16)),7))
newm3 = np.reshape(m3,newshape=(m3.shape[0],int((IMG_SIZE*IMG_SIZE)/(32*32)),7))
newm4 = np.reshape(m4,newshape=(m4.shape[0],int((IMG_SIZE*IMG_SIZE)/(64*64)),7))
allanchors = np.hstack([newm1,newm2,newm3,newm4])
fig,ax = plt.subplots(1,20,figsize=(100,5))
for pi in range(20):
  img_to_show = x_train[pi]
  ax[pi].imshow(img_to_show)
  boxes = allanchors[pi][allanchors[pi][:,5]==1]*IMG_SIZE
  for box in range(len(boxes)):
    ax[pi].add_patch(patches.Rectangle((boxes[box,0],boxes[box,1]),boxes[box,2]-boxes[box,0],boxes[box,3]-boxes[box,1],linewidth=1,edgecolor='r',facecolor='none'))
plt.show()

####Test Pre</b>

In [ ]:
from matplotlib import pyplot as plt
from PIL import Image,ImageOps,ImageFilter
import matplotlib.patches as patches
import random
import cv2
from google.colab.patches import cv2_imshow

face_cascade = cv2.CascadeClassifier('haarcascade_frontalface_default.xml')
profile_cascade = cv2.CascadeClassifier('haarcascade_profileface.xml')
eye_cascade = cv2.CascadeClassifier('haarcascade_eye.xml')
item_num = random.randint(0,(x_ou[0].shape[0])-1)
#item_num = 24
img = Image.fromarray((x_ou[0][item_num]).astype('uint8'),mode='RGB')
print(item_num)

fig = plt.figure()
ax = fig.add_subplot(1,1,1)
ticks = np.arange(0, 256, 32)

ax.set_xticks(ticks)
ax.set_yticks(ticks)
ax.grid(b=True,color='black')
gray = cv2.cvtColor(np.array(img), cv2.COLOR_RGB2GRAY)

faces = face_cascade.detectMultiScale(gray, 1.01, 10)
for (x1,y1,w1,h1) in faces:
    print('x1 '+str(x1)+' y1 '+str(y1)+' w1 '+str(w1)+' h1 '+str(h1))
    img = cv2.rectangle(np.array(img),(x1,y1),(x1+w1,y1+h1),(255,0,0),2)
    roi_color = img[y1:y1+h1, x1:x1+w1]

profiles = profile_cascade.detectMultiScale(gray, 1.01, 10)
for (x2,y2,w2,h2) in profiles:
    print('x2 '+str(x2)+' y2 '+str(y2)+' w2 '+str(w2)+' h2 '+str(h2))
    img = cv2.rectangle(np.array(img),(x2,y2),(x2+w2,y2+h2),(0,255,0),2)
    roi_color = img[y2:y2+h2, x2:x2+w2]

ax.imshow(img)

####Model</b>

16 x 16 x 5 x 2

In [ ]:
import requests
from io import BytesIO
import numpy as np
from PIL import Image
from matplotlib import pyplot as plt


response = requests.get('https://www.bmw.co.uk/bmw-cars/explore-the-range/lifestyle/image-thumb__14913__480x270/3-series-1920x1080.jpeg?1539010591')
img = Image.open(BytesIO(response.content))
img2 = img.resize((32, 32), Image.NEAREST) 
img3 =np.expand_dims(img2, axis=0)
print(img3.shape)
p = current_model.predict(img3)
print(p.shape)
print(p[0])

In [ ]:
def fetch_model(input_size):
  files = os.listdir('/drive/My Drive/Colab Notebooks/AnalyticsVidya/FaceDetection')

  matchs = [s for s in files if SAVED_MODEL in s]
  
  least_val_loss = 1000.00
  previous_model_file = ''
  for m in matchs:
      if float((m.split('_',2)[2]).split('.hdf5',1)[0]) < least_val_loss:
        least_val_loss = float((m.split('_',2)[2]).split('.hdf5',1)[0])
        previous_model_file = '/drive/My Drive/Colab Notebooks/AnalyticsVidya/FaceDetection' + m
  if(least_val_loss < 1000.00):
      print("Loading previous model with least val loss: "+ str(previous_model_file))
      current_model = build_model(img_size=input_size)
      current_model.load_weights(previous_model_file)
  else:
      print("No previous model..   creating new Model")
      current_model = build_model(img_size=input_size)
  
  return current_model

In [ ]:
from tensorflow.python.ops import clip_ops
from tensorflow.python.framework import constant_op


def _constant_to_tensor(x, dtype):
    return constant_op.constant(x, dtype=dtype)


with strategy.scope():

    num_classes = len(CLASSES)
    
    # About why we set `reduction` to 'none', please check this tutorial
    # https://www.tensorflow.org/tutorials/distribute/custom_training#define_the_loss_function
    # In particular, read the paragraph
    #     <<If using tf.keras.losses classes (as in the example below), the loss reduction needs to be explicitly specified to be one of NONE or SUM. AUTO and SUM_OVER_BATCH_SIZE are disallowed when used with tf.distribute.Strategy.>>
    
    loss_object = tf.keras.losses.CategoricalCrossentropy(from_logits=False, reduction='none', label_smoothing=0.1)

    def loss_function(labels, prob_dists, sample_weights=None):

        if not sample_weights:
            sample_weights = 1.0

        # While trained with BATCH_SIZE = 8 * strategy.num_replicas_in_sync, I got `nan` values.
        # Since we pass probability distribution to `CategoricalCrossentropy` with `from_logits` = False,
        # which has numerical unstability issue,
        # we use the same trick in the source code to avoid such unstabiltiy.
        epsilon_ = _constant_to_tensor(tf.keras.backend.epsilon(), prob_dists.dtype.base_dtype)
        prob_dists = clip_ops.clip_by_value(prob_dists, epsilon_, 1 - epsilon_)
        
        labels = tf.keras.backend.one_hot(labels, num_classes)
        
        loss = loss_object(labels, prob_dists)
        
        # About why we use `tf.nn.compute_average_loss`, please check this tutorial
        # https://www.tensorflow.org/tutorials/distribute/custom_training#define_the_loss_function
        
        loss = tf.nn.compute_average_loss(loss, global_batch_size=BATCH_SIZE)

        return loss